In [1]:
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import string
import plotly
import plotly.graph_objs as go
import jieba
import wordcloud
from collections import Counter
def load_stopword(stop_word_file):
    #stop_word_file = "F:\\Desktop\\NLP_聊天\\stopwords.txt"
    stop_word = open(stop_word_file,encoding="utf-8").read().split('\n')
    return stop_word
def get_time(timestamp):
    return datetime.fromtimestamp(timestamp)
stop_word = load_stopword("D:\\Desktop\\wechat_dig\\stopwords.txt")
def remove_word(all_seg_word,stop_word=stop_word):
    remove_str = string.punctuation + string.ascii_letters + string.digits + string.whitespace
    new_seg_word = []
    if stop_word is not None:
        all_seg_word = [_ for _ in all_seg_word if _ not in stop_word]
    for word in all_seg_word:
        if word in ["我","你","他","她","的","了","不是", "就是", "这么", "怎么", "这个", "不能", "什么", "没有","emoji","。","…","，","\t","\n"]:
            continue
        elif word in list("是也去吗吧不"):
            continue
        elif word in ["可以","我们","你们","觉得","他们","流泪","捂脸","一个","飞吻","微笑","难过","发呆","咒骂","尴尬","冷汗","大哭","再见","害羞","尴尬","冷汗","奸笑","可怜","一下","所以","还是",
                     "那个","现在","然后","可能","知道","大家","自己","但是","真的","流汗","还有","应该",'一条',"感觉"]:
            continue
        elif "哈" in word:
            continue
        elif word in remove_str:
            continue
        elif not set(word).difference(set(remove_str)):
            continue
        new_seg_word.append(word.strip())
    return new_seg_word


In [ ]:

message_odir = "D:\\Desktop\\微信导出\\关爱博士成长㍿_3127835014@chatroom"

message_path = message_odir + "\\js\\message.js"
data = {}
message = open(message_path,'r',encoding='utf-8').read()
exec(message.replace('var ',''))

# chartroom ID
gID = data["owner"]['user']
gname = data["owner"]['name']
# For chatroom
# get all user ID and its name
members = data["member"]
mid2username = {mid: members[mid]['name'] for mid in members}
mid2username.pop(gID)

In [ ]:
# Member id to user name
mid2username

In [4]:
all_messages_df = pd.DataFrame.from_records(data['message'])


all_messages_df.loc[:,'fromWho'] = [mid2username.get(record,'碎碎念达人')
                                    for record in all_messages_df.loc[:,"m_nsRealChatUsr"]]
all_messages_df.loc[:,'datetime'] = [get_time(record)
                                     for record in all_messages_df.loc[:,"m_uiCreateTime"]]

all_messages_df= all_messages_df.loc[:,["fromWho","datetime","m_nsContent"]]
all_messages_df = all_messages_df.set_index("datetime")

# basic info

In [ ]:
mid2username

In [ ]:
# number of message (each one)
from collections import Counter
Counter(mid2username.get(_['m_nsRealChatUsr'],'碎碎念达人')
        for _ in data['message'])

# fig1

In [ ]:
fig = go.Figure()
for sid in set(all_messages_df.loc[:,"fromWho"]):
    sub_df = all_messages_df.loc[all_messages_df.loc[:,'fromWho'] == sid,:]
    sub_data = sub_df.resample('M').count()
    fig.add_scatter(x=sub_data.index,
                    y= sub_data.loc[:,'m_nsContent'],
                    name=sid)
plotly.offline.iplot(fig)

# fig2 

In [8]:
text_data = all_messages_df.resample('Q').sum()
count_data = all_messages_df.resample('Q').count()

In [ ]:
ys = []
count_num=[]
for t,row in tqdm(text_data.iterrows(),total=text_data.shape[0]):
    thistext = row["m_nsContent"]
    seg_list = jieba.lcut(str(thistext))
    filtered_seg = remove_word(seg_list)
    w = wordcloud.WordCloud(collocations=False)
    w.generate(' '.join(filtered_seg))
    count_dict = w.words_
    ys.append(';'.join([_[0] for _ in sorted(count_dict.items(),
                     key=lambda x:x[1],
                     reverse=True)
                       ][:5]
                      )
             )
    count_num.append(count_data.loc[t,"m_nsContent"])

  0%|                                                                                           | 0/21 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\liaoth\AppData\Local\Temp\jieba.cache
Loading model cost 0.591 seconds.
Prefix dict has been built succesfully.
100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:46<00:00,  2.22s/it]


In [ ]:
fig = go.Figure()
fig.add_scatter(x=text_data.index,
                y=count_num,
                text=ys,
                mode='markers+lines+text',
               textposition='top center')
plotly.offline.iplot(fig)

# wordcloud 1

In [ ]:
total_seg_word = jieba.lcut(' '.join(all_messages_df.loc[:,'m_nsContent']))
filtered_seg_word = remove_word(total_seg_word)

In [ ]:
import wordcloud
txt = ' '.join(filtered_seg_word)
w = wordcloud.WordCloud(font_path = 'msyh.ttc' , #使用微软雅黑字体
                        width = 2000, 
                        height = 1400, 
                        background_color = 'white',
                        collocations=False,
                        colormap ='cividis',
                       max_words =500) 
w.generate(txt)
w.to_file(message_odir + '\\total.png')

# wordcloud 2

In [ ]:
year_text_data = all_messages_df.resample('Y').sum()

In [ ]:
ys = []
count_num=[]
for t,row in tqdm(year_text_data.iterrows(),
                  total=year_text_data.shape[0]):
    thistext = row["m_nsContent"]
    seg_list = jieba.lcut(str(thistext))
    filtered_seg = remove_word(seg_list)
    w = wordcloud.WordCloud(font_path = 'msyh.ttc' ,#使用微软雅黑字体
                            width = 2000, 
                            height = 1400, 
                            max_words =300,
                            background_color = 'white',
                            colormap ='cividis',
                           collocations=False) 
    w.generate(' '.join(filtered_seg))
    os.makedirs(message_odir +'\\年度热词',exist_ok=True)
    w.to_file(message_odir+'\\年度热词' + '\\%s.png' % str(t.year))
